In [1]:
from glob import glob
from ddl_parser import DDL2AlembicParser

## Tables

In [9]:
files_list = [
    'btg_fl_retailer_assgnt.txt',
    'btg_pbc_retailer_assgnt.txt',
    'btg_pbc_route_data.txt',
    'btg_pep_calendar.txt',
    'btg_ref_calendar.txt',
    'fl_cust.txt',
    'fl_prod.txt',
    'fl_sales_2018.txt',
    'fl_sales_2019.txt',
    'fl_sales_2020.txt',
    'fl_sales_2021.txt',
    'gsc_msa_demo.txt',
    'mdm_msa_fl_xref.txt',
    'mdm_msa_pbc_xref.txt',
    'new_pbc_mkt_loc_struct.txt',
    'pbc_cust.txt',
    'pbc_prod.txt',
    'pbc_sales_2019.txt',
    'pbc_sales_2020.txt',
    'pbc_sales_2021.txt',
]

In [15]:
tables_list = [file.split('.')[0] for file in files_list]

tables_list

['btg_fl_retailer_assgnt',
 'btg_pbc_retailer_assgnt',
 'btg_pbc_route_data',
 'btg_pep_calendar',
 'btg_ref_calendar',
 'fl_cust',
 'fl_prod',
 'fl_sales_2018',
 'fl_sales_2019',
 'fl_sales_2020',
 'fl_sales_2021',
 'gsc_msa_demo',
 'mdm_msa_fl_xref',
 'mdm_msa_pbc_xref',
 'new_pbc_mkt_loc_struct',
 'pbc_cust',
 'pbc_prod',
 'pbc_sales_2019',
 'pbc_sales_2020',
 'pbc_sales_2021']

## Convert: DDL File --> Alembic

In [8]:
# read the file
file_contents = open('sql/btg_create_tables.txt').read()

for ddl in file_contents.split(';'):
    print()
    print(DDL2AlembicParser().convert(ddl, tab_spacing='    '))
    
##len([DDL2AlembicParser().convert(ddl, tab_spacing='    ') for ddl in file_contents.split(';')])


op.create_table(
    'btg_fl_cust_v',
    sa.Column('cust_nbr', sa.INT()),
    sa.Column('ctry_code', sa.VARCHAR(3)),
    sa.Column('ctry_descp', sa.VARCHAR(220)),
    sa.Column('co_code', sa.VARCHAR(3)),
    sa.Column('co_descp', sa.VARCHAR(220)),
    sa.Column('rt_nbr', sa.INT()),
    sa.Column('chain_node_sys_nbr', sa.VARCHAR(8)),
    sa.Column('cust_loc_nbr', sa.INT()),
    sa.Column('bus_loc_str_addr1', sa.VARCHAR(35)),
    sa.Column('bus_loc_str_addr2', sa.VARCHAR(35)),
    sa.Column('bus_loc_city', sa.VARCHAR(20)),
    sa.Column('bus_loc_cnty_sburb', sa.VARCHAR(20)),
    sa.Column('bus_loc_st_prov', sa.VARCHAR(20)),
    sa.Column('bus_loc_ctry', sa.VARCHAR(20)),
    sa.Column('bus_loc_post_code', sa.VARCHAR(9)),
    sa.Column('bus_loc_post_code_zip5', sa.VARCHAR(5)),
    sa.Column('contc_persn_nm', sa.VARCHAR(40)),
    sa.Column('cust_phone_nbr', sa.VARCHAR(15)),
    sa.Column('cust_fax_nbr', sa.VARCHAR(15)),
    sa.Column('contr_id', sa.VARCHAR(20)),
    sa.Column('prmy_cust_n

### Read all Files

for filename in glob('sql/*.txt'):
    print("#"*100)
    print(filename)
    print("#"*100)
    
    # read the file
    file_contents = open(filename).read()
    
    for ddl in file_contents.split(';'):
        print()
        print(DDL2AlembicParser().convert(ddl, tab_spacing='    '))